In [1]:
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


Source_data = pd.read_csv("/mnt/c/Users/ralvin/Downloads/837I00111182024.csv")


print(Source_data.shape)


Source_data.head()


#### 109 unique claim Ids
#### 91 unique claim numbers
#### 59 providers

(446, 42)


,Claim ID,Claim Number,Line ID,TIN,Provider,Address,City,State,Zip,Patient ID,Patient,Date of Birth,Patient Group/Policy Number,DRG,Diag Code 1st,Diag Code 2nd,Diag Code 3rd,Diag Code 4th,Begin DOS,End DOS,Revenue Code,HCPCS/CPT Code,Mod,Mod (2nd),Units,Billed Amount,MAR,IsQPA,Billing Provider Taxonomy,JurisdictionState,4LCodes,NPI,PlaceofService,AnesthesiaUnits,Billtype,BillProvAddress1,BillProvAddress2,BillProvCity,BillProvState,BillProvZip,DRGCode,ClientName
0,A3038B5E4B6C914tksft,4.32E+14,1,941105628,KAISER FOUNDATION HOSPITALS,FILE 54602,LOS ANGELES,CA,900749998,569713835,WATKINS MONIC,8/12/1982,NNMCMC,NaN,M25562,NaN,NaN,NaN,9/26/2024,9/26/2024,320,73564,NaN,NaN,1,987.0,156.72,N,282N00000X,CA,NaN,NaN,13,,13,FILE 54602,,LOS ANGELES,CA,900749998,,NovaNet - MCMC - RHP
1,BE9124A610A9998tksft,CLU20241115325204,1,351166081,JOHNSON MEMORIAL HOSPITAL,1125 W JEFFERSON STREET,FRANKLIN,IN,461312140,QWCR14517,FISHER TERRI,1/11/1974,100600,NaN,M7672,M722,M7732,NaN,10/17/2024,10/17/2024,510,G0463,NaN,NaN,1,65.0,65.00,N,282N00000X,IN,NaN,1.578930e+09,13,,13,1125 W JEFFERSON STREET,,FRANKLIN,IN,461312140,,Lucent - RHP
2,317D671330C4442tksft,U424319281216,1,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,278,C1889,NaN,NaN,1,3.0,3.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
3,317D671330C4442tksft,U424319281216,2,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,300,81025,NaN,NaN,1,54.0,54.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
4,317D671330C4442tksft,U424319281216,3,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88305,59,NaN,4,1000.0,1000.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP


In [4]:
claims_ids = list(Source_data['Claim ID'])
lines = list(Source_data['Claim Number'])
provider = list(Source_data['TIN'])

len(set(claims_ids))
len(set(lines))
len(set(provider))


59

In [ ]:
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


Source_data = pd.read_csv("/mnt/c/Users/ralvin/Downloads/837I00111182024.csv")


print(Source_data.shape)


Source_data.head()


#### 109 unique claim Ids
#### 91 unique claim numbers
#### 59 providers

In [34]:





def salesforce_connection(sandbox=True):
    session = requests.Session()
    security_token = 'pIjwc8P0OB2ZisbQYhlKxfLd'
    username = 'ralvin@relianthp.com.tctdev'
    password = 'Flash363!'
    domain = 'test'

    session_id, instance = SalesforceLogin(username=username, password=password, security_token=security_token, domain=domain)
    sf = Salesforce(instance=instance, session_id=session_id, session=session)
    print('Connected to Salesforce (Sandbox)')
    return sf

sf = salesforce_connection()

Connected to Salesforce (Sandbox)


In [35]:
all_objects = sf.describe()["sobjects"]


claims_metadata = sf.Claims__c.describe()
claim_cols = [field['name'] for field in claims_metadata['fields']]

line_item_metadata = sf.Line_Items__c.describe()
line_item_cols = [field['name'] for field in line_item_metadata['fields']]

provider_tin_metadata = sf.Provider_TIN__c.describe()
provider_tin_cols = [field['name'] for field in provider_tin_metadata['fields']]

jurisdiction_metadata = sf.Jurisdiction__c.describe()
jurisdiction_cols = [field['name'] for field in jurisdiction_metadata['fields']]

group_clients_metadata = sf.Groups_Clients__c.describe()
group_clients_cols = [field['name'] for field in group_clients_metadata['fields']]

drg_metadata = sf.DRG__c.describe()
drg_cols = [field['name'] for field in drg_metadata['fields']]

hcpcs_cpt_metadata = sf.HCPCS_CPT_Code__c.describe()
hcpcs_cpt_cols = [field['name'] for field in hcpcs_cpt_metadata['fields']]

accounts_metadata = sf.Account.describe()
account_cols = [field['name'] for field in accounts_metadata['fields']]

clients_metadata = sf.Clients__c.describe()
clients_cols = [field['name'] for field in clients_metadata['fields']]

provider_specialty_metadata = sf.Provider_Specialty__c.describe()
provider_specialty_cols = [field['name'] for field in provider_specialty_metadata['fields']]

needed_objects = { "Claims__c": claim_cols, "Line_Items__c": line_item_cols, "Provider_TIN__c": provider_tin_cols, 
                 'Jurisdiction__c': jurisdiction_cols, 'Groups_Clients__c': group_clients_cols, 'DRG__c': drg_cols, 
                 'HCPCS_CPT_Code__c': hcpcs_cpt_cols, 'Account':account_cols, 'Clients__c': clients_cols, 'Provider_Specialty__c':provider_specialty_cols}

for table in needed_objects:
    print(table)
    print(needed_objects[table])
    print(len(needed_objects[table]))

table_store = {}

for object_name in needed_objects:
    print(object_name)

    object_fields = needed_objects[object_name]

    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)
    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')  # Remove unwanted attributes column
    print(df.shape)
    table_store[object_name] = df

Claims__c
['Id', 'IsDeleted', 'Name', 'CreatedDate', 'CreatedById', 'LastModifiedDate', 'LastModifiedById', 'SystemModstamp', 'LastActivityDate', 'LastViewedDate', 'LastReferencedDate', 'Duplicate__c', 'Client__c', 'Claim_Number__c', 'Jurisdiction__c', 'Provider_TIN__c', 'Contains_Line_Items__c', 'Status__c', 'Appeal__c', 'Date_Fee_Paid__c', 'Fee_Credit_Date__c', 'Diag_Code__c', 'Diag_Code_2nd__c', 'Diag_Code_3rd__c', 'Proc_Code__c', 'Proc_Code_2nd__c', 'Diag_Code_4th__c', 'Total_Billed__c', 'Provider__c', 'Group_Client__c', 'Claim_ID__c', 'Received_Date__c', 'Appeal_Rec_d_Date__c', 'Last_Contact__c', 'Appeal_Stage__c', 'State_Appeal__c', 'Contact__c', 'Phone__c', 'Fax__c', 'Email__c', 'Notes__c', 'Appeal_Closed_Date__c', 'POS__c', 'Attorney_Assigned__c', 'Detailed_Appeal_Resp__c', 'Begin_DOS__c', 'End_DOS__c', 'Total_MAR__c', 'Total_Allowed__c', 'Total_Reductions__c', 'Fee__c', 'Patient__c', 'Date_of_Birth__c', 'Patient_ID__c', 'Total_Additional_Payment__c', 'Fee_Credit__c', 'Savings_

In [2]:
########### tables to update: Provider_TIN__c, Account, Claims__c, Line_Items__c


#notes:Flagged is constant field under claims table .. should be in config file "to be repriced" is default, FMP is constant field under Line table


Provider_TIN_dict = {'MAPPINGS':{'TIN':['Name', 'Unique_TIN__c'], 'Address':['Address__c'], 'City':['City__c'], 'State':['State__c'], 'Zip':['Zip__c'],
                      'Provider':['Provider_Name__c']}, 'KEYS':{'Provider_TIN__c':'Unique_TIN__c'}}

Account_dict = {'MAPPINGS':{'TIN':['Tax_ID__c'], 'BillProvAddress1':['BillingAddress'], 'BillProvCity':['BillingCity'], 'BillProvState':['BillingState'],
                 'BillProvZip':['BillingPostalCode'], 'Provider':['Name']}, 'KEYS':{'Account':'Tax_ID__c'}}

Claim_dict = {'MAPPINGS':{'Claim ID':['Claim_ID__c'], 'TIN':['Provider_TIN__c'], 'Patient Group/Policy Number':['Group_Client__c'], 'Claim Number':['Claim_Number__c'],'JurisdictionState':['Jurisdiction__c'],
               'Diag Code 1st':['Diag_Code__c'],'Diag Code 2nd':['Diag_Code_2nd__c'], 'Diag Code 3rd':['Diag_Code_3rd__c'], 'Diag Code 4th':['Diag_Code_4th__c'], 'Provider':['Provider__c'],
               'Patient':['Patient__c'], 'Patient ID':['Patient_ID__c'], 'Date of Birth':['Date_of_Birth__c'],'Zip':['Provider_Zip__c'], 'DRG':['DRG__c'], 'IsQPA':['QPA__c'], 'NPI':['NPI__c'],
               'Begin DOS':['Begin_DOS__c'], 'End DOS':['End_DOS__c'], 'PlaceofService':['POS__c'], 'Billing Provider Taxonomy':['Provider_Specialty__c', 'Service_Type__c']}, 'KEYS':{'Claims__c':'Claim_ID__c'}, 
               'TABLE_CONVERT':{'TIN':'Provider_TIN__c', 'Patient Group/Policy Number':'Groups_Clients__c', 'JurisdictionState':'Jurisdiction__c', 'DRG':'DRG__c', 'Billing Provider Taxonomy':'Provider_Specialty__c' }
               }

# Line_item_dict = {'Begin DOS':['DOS__c'], 'Revenue Code':['Rev_Code__c'],'Billed Amount':['Billed_Amount__c'], 'Claim ID':['RHP_ID__c'], 'Mod':['Mod__c'],
#                    'Mod (2nd)':['Mod_2nd__c'], 'Units':['Units__c'], 'MAR':['MAR__c'], 'HCPCS/CPT Code':['HCPCS_CPT_Code__c'], 'Line ID':['Line_ID__c'], 
#                     '4LCodes':['Exp_Code_2nd__c']}

In [37]:
table_store['DRG__c'].head()


,Id,OwnerId,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Description__c,DRG_Weight__c,NJ_DRG_QPA__c,TN_DRG_QPA__c,MO_DRG_QPA__c,NJ_Per_Diem_QPA__c,TN_Per_Diem_QPA__c,MO_Per_Diem_QPA__c
0,a014W00000ym0QaQAI,0054W00000ASrc5QAD,False,224,2020-10-06T20:20:49.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,None,None,None,CARDIAC DEFIB IMPLANT W CARDIAC CATH W/O AMI/H...,4.8478,151205.55,98362.65,82560.82,21267.60,13831.03,11612.11
1,a014W00000ym0QbQAI,0054W00000ASrc5QAD,False,225,2020-10-06T20:20:49.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,None,None,None,CARDIAC DEFIB IMPLANT W CARDIAC CATH W/O AMI/H...,6.9024,107875.68,70175.59,58901.97,28090.47,18273.47,15340.00
2,a014W00000ym0QcQAI,0054W00000ASrc5QAD,False,226,2020-10-06T20:20:49.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,None,None,None,CARDIAC DEFIBRILLATOR IMPLANT W/O CARDIAC CATH...,5.3863,136524.80,88812.49,74544.88,23337.68,15182.93,12744.19
3,a014W00000ym0QdQAI,0054W00000ASrc5QAD,False,227,2020-10-06T20:20:49.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,None,None,None,CARDIAC DEFIBRILLATOR IMPLANT W/O CARDIAC CATH...,4.4776,107086.51,69662.21,58471.07,34994.28,22763.85,19108.78
4,a014W00000ym0QeQAI,0054W00000ASrc5QAD,False,228,2020-10-06T20:20:49.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,0054W00000ASrc5QAD,2023-10-24T18:28:32.000+0000,None,None,None,OTHER CARDIOTHORACIC PROCEDURES W MCC,9.4790,105115.71,68380.16,57394.98,14168.86,9215.92,7737.43


In [38]:
def populate_df(source_dict, Source_data, table_store):
    new_columns = [col for mapped_cols in source_dict['MAPPINGS'].values() for col in mapped_cols]
    new_df = pd.DataFrame(columns=new_columns)

    try:
        Table_convert_cols = list(source_dict['TABLE_CONVERT'].keys())
    except:
        Table_convert_cols = []

    print(Table_convert_cols)

    for source_col, target_cols in source_dict['MAPPINGS'].items():
        if source_col in Table_convert_cols:
            print(source_col)
            map_table = source_dict['TABLE_CONVERT'][source_col]
            print(map_table)
            print('')
            if source_col == 'TIN':
                string_tin_ids= [str(tin_id) for tin_id in Source_data[source_col]]
                Source_data[source_col] = string_tin_ids
                merged = Source_data.merge(table_store[map_table].loc[:, ['Id', 'Name']], left_on='TIN', right_on='Name', how='left')
                for target_col in target_cols:
                    new_df[target_col] = merged['Id']
       
            if source_col == 'Patient Group/Policy Number':
                string_ids= [str(ids) for ids in Source_data[source_col]]
                Source_data[source_col] = string_ids
                merged = Source_data.merge(table_store[map_table].loc[:, ['Id', 'Group_Number__c']], left_on=source_col, right_on='Group_Number__c', how='left')
                for target_col in target_cols:
                    new_df[target_col] = merged['Id']
                    
            if source_col == 'JurisdictionState':
                string_ids= [str(ids) for ids in Source_data[source_col]]
                Source_data[source_col] = string_ids
                merged = Source_data.merge(table_store[map_table].loc[:, ['Id', 'Name']], left_on=source_col, right_on='Name', how='left')
                for target_col in target_cols:
                    new_df[target_col] = merged['Id']

            if source_col == 'DRG':
                # string_ids= [int(ids) for ids in Source_data[source_col]]
                

                string_ids = [
                    int(ids) if pd.notna(ids) and str(ids).isdigit() else None 
                    for ids in Source_data[source_col]
                ]
                Source_data[source_col] = string_ids

                merged = Source_data.merge(table_store[map_table].loc[:, ['Id', 'Name']], left_on=source_col, right_on='Name', how='left')
                for target_col in target_cols:
                    new_df[target_col] = merged['Id']
        
        elif source_col not in Table_convert_cols:
            if source_col in Source_data.columns:
                for target_col in target_cols:
                    new_df[target_col] = Source_data[source_col]
            


    keys = source_dict['KEYS']

    return new_df, keys


new_df, keys = populate_df(Claim_dict,Source_data,table_store)

['TIN', 'Patient Group/Policy Number', 'JurisdictionState', 'DRG', 'Billing Provider Taxonomy']
TIN
Provider_TIN__c

Patient Group/Policy Number
Groups_Clients__c

JurisdictionState
Jurisdiction__c

DRG
DRG__c

Billing Provider Taxonomy
Provider_Specialty__c



In [42]:
# new_df = new_df.loc[:, ['Claim_ID__c', 'Provider_TIN__c', 'check_tinID']]
# provider_data_pull[provider_data_pull['Unique_TIN__c'].isin(string_tin_ids_testData)].shape ##29


# new_df


# # Filter rows where 'Provider_TIN__c' is not missing
# filtered_df = new_df.loc[new_df['Provider_TIN__c'].notna(), ['check_tinID']]

# # Count unique 'check_tinID' values
# unique_count = filtered_df['check_tinID'].nunique()

# print(f"Count of unique 'check_tinID' where 'Provider_TIN__c' is not missing: {unique_count}")


In [12]:
new_df[~new_df['Provider_TIN__c'].isna()].shape

(0, 22)

In [67]:
table_store['Provider_TIN__c'].head()

,Id,OwnerId,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Special_Repricing__c,Contact__c,Phone__c,Fax__c,Email__c,Notes__c,Unique_TIN__c,Address__c,City__c,State__c,Zip__c,Shelby_Co_Contract__c,City_of_Covington_Contract__c,Provider_Name__c,Avg_WC_Savings__c
0,a0F4W00000W1GxyUAF,0054W00000ASrc5QAD,False,000000002,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,2024-12-06T15:27:38.000+0000,2024-12-06T15:27:38.000+0000,False,None,None,None,None,None,000000002,PO Box 362,Royal Oak,MI,480680000,None,None,Provider Miscellaneous,0.0
1,a0F4W00000W1GxzUAF,0054W00000ASrc5QAD,False,000000009,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000009,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,VISION EMPLOYEE CLAIMS,0.0
2,a0F4W00000W1Gy0UAF,0054W00000ASrc5QAD,False,00000002,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,00000002,None,None,None,None,None,None,MISCELLANEOUS PROVIDER,0.0
3,a0F4W00000W1Gy1UAF,0054W00000ASrc5QAD,False,000000021,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000021,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,MEDICAL CLAIM,0.0
4,a0F4W00000W1Gy2UAF,0054W00000ASrc5QAD,False,000000022,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000022,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,DENTAL CLAIMS,0.0


In [1]:
# dictionaries = [Provider_TIN_dict, Account_dict, Claim_dict, Line_item_dict]
dictionaries = [Claim_dict]


def populate_df(source_dict, Source_data):
    new_columns = [col for mapped_cols in source_dict['MAPPINGS'].values() for col in mapped_cols]
    new_df = pd.DataFrame(columns=new_columns)


    for source_col, target_cols in source_dict['MAPPINGS'].items():
        if source_col in Source_data.columns:
            for target_col in target_cols:
                new_df[target_col] = Source_data[source_col]

    keys = source_dict['KEYS']

    return new_df, keys


def updated_data_pull(object_name, needed_objects):

    object_fields = needed_objects[object_name]
    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)

    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')  # Remove unwanted attributes column
    
    return df


def upsert(new_df, keys):

    object_name = list(keys.keys())[0]
    sf_object = getattr(sf, object_name)
    object_key = list(keys.values())[0]

    print(object_key)
   

    for i in range(len(new_df)):
        print(i)
        row = new_df.iloc[i, :].to_dict()
        upsert_dict = {k: v for k, v in row.items() if k != object_key}
    
        print(upsert_dict)
        try:
            # sf_object.upsert(f"{object_key}/{row[object_key]}", upsert_dict)
            pass
        except Exception as e:
            print(f"Error during upsert: {e}")

        break



for dict in dictionaries:
    new_df, keys = populate_df(dict, Source_data)
    upsert_data = upsert(new_df, keys)
    # updated_providers_table = updated_data_pull('Provider_TIN__c', needed_objects)
    # print(updated_providers_table.shape)
    


    
    ###upsert_data

    break

NameError: name 'Claim_dict' is not defined

In [17]:
provider_data_pull = table_store['Provider_TIN__c']
new_tin_ids_testData = list(Source_data['TIN'])
string_tin_ids_testData = [str(tin_id) for tin_id in new_tin_ids_testData]
provider_data_pull[provider_data_pull['Name'].isin(string_tin_ids_testData)].shape

(29, 27)

In [44]:
provider_data_pull = table_store['Provider_TIN__c']

new_tin_ids_testData = list(new_df['Unique_TIN__c'])
string_tin_ids_testData = [str(tin_id) for tin_id in new_tin_ids_testData]

len(string_tin_ids_testData)

446

In [ ]:
provider_data_pull[provider_data_pull['Unique_TIN__c'].isin(string_tin_ids_testData)].shape



(29, 27)

In [36]:
merged = Source_data.merge(provider_data_pull.loc[:, ['Id', 'Name']], left_on='TIN', right_on='Name', how='left')
merged.loc[:, ['Id', 'Name', 'TIN']].drop_duplicates(subset='TIN')

,Id,Name,TIN
0,a0FVF000001zh5t2AA,941105628,941105628
1,a0F4W00000W1KI4UAN,351166081,351166081
2,a0FVF000001f3W22AI,610461753,610461753
11,NaN,NaN,943377073
21,NaN,NaN,590657322
23,a0F4W00000W1LmPUAV,942854057,942854057
32,NaN,NaN,340714357
34,a0FVF000000uPgT2AU,275146101,275146101
35,NaN,NaN,383870608
36,NaN,NaN,352072586


In [ ]:
##### total shape of provider table after upsert should be 10823 + 446(new data cases) - 29(records already up)

In [ ]:



new_columns = [col for mapped_cols in Provider_TIN_dict.values() for col in mapped_cols]
new_df = pd.DataFrame(columns=new_columns)

# Populate the new DataFrame by mapping values
for source_col, target_cols in Provider_TIN_dict.items():
    if source_col in Source_data.columns:
        for target_col in target_cols:
            new_df[target_col] = Source_data[source_col]

# Display the new DataFrame
new_df.head()


for i in range(len(new_df)):
    print(i)
    row = new_df.iloc[i, :].to_dict()
    upsert_dict = {k: v for k, v in row.items() if k != 'Unique_TIN__c'}
    print(row)

In [18]:
sorted(table_store['Line_Items__c'].columns)

['Additional_Payment__c',
 'Allowed__c',
 'Bill_Type__c',
 'Billed_Amount__c',
 'Client__c',
 'Code_Edit__c',
 'CreatedById',
 'CreatedDate',
 'DOS__c',
 'Exp_Code_2nd__c',
 'Exp_Code__c',
 'Exp_Desc_2nd__c',
 'Exp_Desc__c',
 'HCPCS_CPT_Code__c',
 'Id',
 'IsDeleted',
 'Jurisdiction_Claim__c',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDate',
 'LastViewedDate',
 'Line_ID__c',
 'Line_Number__c',
 'MAR__c',
 'MO_Anes_QPA_Reductions__c',
 'MO_Anes_QPA__c',
 'MO_Fac_QPA_Allowed__c',
 'MO_Fac_QPA_Reductions__c',
 'MO_QPA_Allowed__c',
 'MO_QPA_Reductions__c',
 'Mod_2nd__c',
 'Mod__c',
 'NDC_Number__c',
 'Name',
 'Pricing_Method__c',
 'Pricing_Trigger__c',
 'Provider_TIN_Claim__c',
 'QPA_Allowed__c',
 'QPA_Reductions__c',
 'RHP_ID__c',
 'Reconsideration__c',
 'Reductions__c',
 'Rev_Code__c',
 'Savings__c',
 'SystemModstamp',
 'Unique_Line_Id__c',
 'Units__c']

In [19]:
sorted(table_store['Claims__c'].columns)

['API_Confirmation__c',
 'API_Status__c',
 'Appeal_Closed_Date__c',
 'Appeal_Rec_d_Date__c',
 'Appeal_Stage__c',
 'Appeal__c',
 'Attorney_Assigned__c',
 'Balance_Bill__c',
 'Begin_DOS__c',
 'Bill_Type__c',
 'Claim_ID__c',
 'Claim_Number__c',
 'Client_Id__c',
 'Client__c',
 'Company__c',
 'Contact_Name__c',
 'Contact__c',
 'Contains_Line_Items__c',
 'Contract__c',
 'CreatedById',
 'CreatedDate',
 'Created_Date_Formula__c',
 'DRG__c',
 'Date_Fee_Paid__c',
 'Date_of_Birth__c',
 'Detailed_Appeal_Resp__c',
 'Diag_Code_2nd__c',
 'Diag_Code_3rd__c',
 'Diag_Code_4th__c',
 'Diag_Code__c',
 'Duplicate__c',
 'Email__c',
 'End_DOS__c',
 'Exp_Code_Test2__c',
 'Exp_Code_Test__c',
 'Fax__c',
 'Fee_Credit_Date__c',
 'Fee_Credit__c',
 'Fee__c',
 'Form_Type__c',
 'Gross_Fee__c',
 'Group_Client__c',
 'Id',
 'IsDeleted',
 'Jurisdiction__c',
 'LastActivityDate',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDate',
 'LastViewedDate',
 'Last_Contact__c',
 'NPI__c',
 'Name',
 'Notes__c',
 'Origina

In [22]:
sorted(table_store['Provider_TIN__c'].columns)

['Address__c',
 'Avg_WC_Savings__c',
 'City__c',
 'City_of_Covington_Contract__c',
 'Contact__c',
 'CreatedById',
 'CreatedDate',
 'Email__c',
 'Fax__c',
 'Id',
 'IsDeleted',
 'LastActivityDate',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDate',
 'LastViewedDate',
 'Name',
 'Notes__c',
 'OwnerId',
 'Phone__c',
 'Provider_Name__c',
 'Shelby_Co_Contract__c',
 'Special_Repricing__c',
 'State__c',
 'SystemModstamp',
 'Unique_TIN__c',
 'Zip__c']

In [23]:
sorted(table_store['Account'].columns)

['AccountSource',
 'Account_Acronym__c',
 'AnnualRevenue',
 'BillingAddress',
 'BillingCity',
 'BillingCountry',
 'BillingCountryCode',
 'BillingGeocodeAccuracy',
 'BillingLatitude',
 'BillingLongitude',
 'BillingPostalCode',
 'BillingState',
 'BillingStateCode',
 'BillingStreet',
 'Billing_Address__City__s',
 'Billing_Address__CountryCode__s',
 'Billing_Address__GeocodeAccuracy__s',
 'Billing_Address__Latitude__s',
 'Billing_Address__Longitude__s',
 'Billing_Address__PostalCode__s',
 'Billing_Address__StateCode__s',
 'Billing_Address__Street__s',
 'Billing_Address__c',
 'Claims_System__c',
 'Copy_Billing_Address_to_Shipping_Address__c',
 'CreatedById',
 'CreatedDate',
 'Default_Pricing_Method__c',
 'Description',
 'Email__c',
 'External_ID__c',
 'Fax',
 'Id',
 'Industry',
 'Insurance_Type__c',
 'Integration_Setting_Name__c',
 'IsCustomerPortal',
 'IsDeleted',
 'IsPartner',
 'Jigsaw',
 'JigsawCompanyId',
 'LastActivityDate',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDat